# Récupération des matchs manquants

Suite au merge des datasets nous avons remarqué qu'il manquait un grand nombre de d'observations. 

Certain matchs n'étaient pas dans les datasets play by play. Il est possible de les récupérer via l'API nba_api afin de les réinjecter dans notre pre processing

In [ ]:
# Matchs qui n'apparaissent que dans Shot locations
game_ids = df_merged[df_merged['GAME_ID'].isna()]['Game ID'].unique()
len(game_ids)

1714

In [ ]:
from nba_api.stats.endpoints import playbyplayv2
from tqdm import tqdm

# pour chaque game id de Shot locations non présent dans les play by play
# on récupère les play by play sur le site de la NBA
liste_df = []
for id in tqdm(game_ids):
    string_id = "00" + str(int(id))
    liste_df.append(playbyplayv2.PlayByPlayV2(string_id).get_data_frames()[0])


100%|██████████| 1714/1714 [31:03<00:00,  1.09s/it]


In [ ]:
# concaténation
missing_play_by_play = pd.concat(liste_df)

In [ ]:
# enregistrement du dataset
missing_play_by_play.to_csv("../data/raw/missing_pbp.csv")

# Récupération des stats manquants

In [17]:
import pandas as pd
BEST_PLAYERS =['Kobe Bryant', 'LeBron James', 'Stephen Curry', 'Kevin Durant', 'Dwyane Wade', 
               'Dirk Nowitzki', 'Tim Duncan', "Shaquille O'Neal", "Steve Nash", "Kawhi Leonard", 
               "James Harden", "Jason Kidd", "Allen Iverson", "Chris Webber", "Kevin Garnett", 
               "Paul Pierce", "Giannis Antetokounmpo", "Jimmy Butler", "Russell Westbrook", "Dwight Howard"]


In [20]:
dfs = []
for i in range(2018,2021):
    df = pd.read_csv('../data/raw/season_stats_'+str(i)+'.csv')
    df_advanced = pd.read_csv('../data/raw/season_stats_advanced_'+str(i)+'.csv')
    df['Year']=i
    df = df.merge(df_advanced, on=['Player', 'Tm','Pos', 'Age', 'G', 'MP'])
    dfs.append(df)

stats = pd.concat(dfs)

In [22]:
stats = stats[stats.Player.isin(BEST_PLAYERS)]

In [23]:
stats.columns

Index(['Rk_x', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Player-additional_x', 'Year', 'Rk_y', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'Player-additional_y'],
      dtype='object')

In [24]:
# Change column order to fit existing data
stats = stats.reindex(['Year','Player','Pos','Age','Tm','G','GS','MP',
                       'PER','TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%',
                       'BLK%','TOV%','USG%', 'Unnamed: 19','OWS','DWS','WS','WS/48','Unnamed: 24',
                       'OBPM','DBPM','BPM','VORP','FG','FGA','FG%','3P','3PA','3P%',
                       '2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB',
                       'AST','STL','BLK','TOV','PF','PTS'], axis=1)
    

In [25]:
#save
stats.to_csv('../data/raw/seasons_stats_2018-2020.csv')

In [26]:
stats

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
15,2018,Giannis Antetokounmpo,PF,23,MIL,75,75,2756,27.3,0.598,...,0.760,156,597,753,361,109,106,223,231,2014
93,2018,Jimmy Butler,SG,28,MIN,59,59,2164,23.7,0.590,...,0.854,79,235,314,288,116,24,108,78,1307
146,2018,Stephen Curry,PG,29,GSW,51,51,1631,28.2,0.675,...,0.921,36,225,261,310,80,8,153,114,1346
171,2018,Kevin Durant,SF,29,GSW,68,68,2325,26.0,0.640,...,0.889,31,433,464,366,50,119,207,133,1792
233,2018,James Harden,SG,28,HOU,72,72,2551,29.8,0.619,...,0.858,41,348,389,630,126,50,315,169,2191
281,2018,Dwight Howard,C,32,CHO,81,81,2463,20.5,0.577,...,0.574,255,757,1012,105,48,131,208,249,1347
304,2018,LeBron James,PF,33,CLE,82,82,3026,28.6,0.621,...,0.731,97,612,709,747,116,71,347,136,2251
362,2018,Kawhi Leonard,SF,26,SAS,9,9,210,26.0,0.572,...,0.816,6,36,42,21,18,9,16,9,146
460,2018,Dirk Nowitzki,C,39,DAL,77,77,1900,16.7,0.575,...,0.898,22,416,438,120,43,45,51,146,927
609,2018,Dwyane Wade,SG,36,TOT,67,3,1536,15.0,0.500,...,0.714,50,202,252,228,61,47,140,124,765
